In [15]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [16]:
# AES parameters
NROW = 4
NCOL = 4
NGRID = NROW * NCOL
NBRANCH = NROW + 1     # AES MC branch number

In [17]:
# variable declaration
total_r = 7 # total round
start = 4   # start round, start in {0,1,2,...,total_r-1}
mid = 1  # meet in the middle round, mid in {0,1,2,...,total_r-1}, start != mid
fwd = []    # forward rounds
bwd = []    # backward rounds

if start < mid:
    fwd = list(range(start, mid))
    bwd = list(range(mid + 1, total_r)) + list(range(0, start))
else:
    bwd = list(range(mid + 1, start))
    fwd = list(range(start, total_r)) + list(range(0, mid))

In [18]:
# Declare and initialize model
m = gp.Model('model_%dx%d_%dR_Start_r%d_Meet_r%d' % (NROW, NCOL, total_r, start, mid))
m

<gurobi.Model Continuous instance model_4x4_7R_Start_r4_Meet_r1: 0 constrs, 0 vars, No parameter changes>

In [19]:
# create Encode class to encode coloring scheme
class CellEncode(object):
    def __init__(self, x: gp.Var, y:gp.Var, z:gp.Var, w:gp.Var):
        self.b = x
        self.r = y
        self.g = z
        self.w = w
        
# define variables to represent state pattern
full_state = np.ndarray(shape=(total_r, NROW, NCOL), dtype=CellEncode)
for r in range(total_r):
    for i in range(NROW):
        for j in range(NCOL):
            full_state[r, i, j] = CellEncode(
                m.addVar(vtype=GRB.BINARY, name= "R%d[%d,%d]_b" %(r,i,j)),
                m.addVar(vtype=GRB.BINARY, name= "R%d[%d,%d]_r" %(r,i,j)),
                m.addVar(vtype=GRB.BINARY, name= "R%d[%d,%d]_g" %(r,i,j)),
                m.addVar(vtype=GRB.BINARY, name= "R%d[%d,%d]_w" %(r,i,j))
            )

In [20]:
# assign and align variables for intermediate states
S = np.ndarray(shape=(total_r, NROW, NCOL), dtype= CellEncode)  # store the SB state at each round
M = np.ndarray(shape=(total_r, NROW, NCOL), dtype= CellEncode)  # store the MC state at each round

S = full_state
for r in range(total_r):
    for i in range(NROW):
        for j in range(NCOL):
            M[r, i, j] = S[r, i, (j+i)%NCOL]    # match SB and MC through SR  

In [21]:
# add grey and white constraints
for r in range(total_r):
    for i in range(NROW):
        for j in range(NCOL):
            m.addConstr(S[r,i,j].g == gp.and_(S[r,i,j].b, S[r,i,j].r))  # grey cell
            m.addConstr(S[r,i,j].w - S[r,i,j].g + S[r,i,j].b + S[r,i,j].r == 1) # white cell 

In [22]:
# the degree of freedom at the starting position
start_df_b = np.ndarray(shape = (NROW, NCOL), dtype = gp.Var)
start_df_r = np.ndarray(shape = (NROW, NCOL), dtype = gp.Var)

for i in range(NROW):
    for j in range(NCOL):
        start_df_b[i,j] = m.addVar(vtype=GRB.BINARY, name= "StartDF[%d,%d]_b" %(i,j))
        start_df_r[i,j] = m.addVar(vtype=GRB.BINARY, name= "StartDF[%d,%d]_r" %(i,j))

# consumed degree of freedom along corresponding direction
consumed_fwd = np.ndarray(shape = (total_r, NCOL), dtype = gp.Var)
consumed_bwd = np.ndarray(shape = (total_r, NCOL), dtype = gp.Var)

for r in range(total_r):
    for j in range(NCOL):
        consumed_fwd = m.addVar(vtype=GRB.BINARY, name= "R%dC%d_fwd" %(r,j))
        consumed_bwd = m.addVar(vtype=GRB.BINARY, name= "R%dC%d_bwd" %(r,j))

# Intermediate values for computations on DoM
match_col_df = np.empty(shape=(NCOL), dtype= gp.Var)
for j in range(NCOL):
    match_col_df[j] = m.addVar(lb=0, ub=NROW, vtype=GRB.INTEGER, name="MatchC" + ('[%d]' % j))

In [23]:
# declare variables relate to df
final_df_b = m.addVar(lb=1, vtype=GRB.INTEGER, name="FinalDF_b")
final_df_r = m.addVar(lb=1, vtype=GRB.INTEGER, name="FinalDF_r")
match_df = m.addVar(lb=1, vtype=GRB.INTEGER, name="MatchDF")
obj = m.addVar(lb=1, vtype=GRB.INTEGER, name="Obj")

In [24]:
# declare column-wise indicator variables to encode MC rules
class ColumnEncode(object):  
    def __init__(self, x: gp.Var, y:gp.Var, z:gp.Var):
        self.u = x
        self.x = y
        self.y = z

S_col = np.ndarray(shape = (total_r, NCOL), dtype = ColumnEncode)
M_col = np.ndarray(shape = (total_r, NCOL), dtype = ColumnEncode)

for r in range(total_r):
    for j in range(NCOL):
        S_col[r, j] = ColumnEncode(
            m.addVar(vtype=GRB.BINARY, name= "R%dSB_C%d_u" %(r,j)),
            m.addVar(vtype=GRB.BINARY, name= "R%dSB_C%d_x" %(r,j)),
            m.addVar(vtype=GRB.BINARY, name= "R%dSB_C%d_y" %(r,j)),
        )

        M_col[r, j] = ColumnEncode(
            m.addVar(vtype=GRB.BINARY, name= "R%dMC_C%d_u" %(r,j)),
            m.addVar(vtype=GRB.BINARY, name= "R%dMC_C%d_x" %(r,j)),
            m.addVar(vtype=GRB.BINARY, name= "R%dMC_C%d_y" %(r,j)),
        )

In [25]:
type(S_col[0,0].u)

gurobipy.Var

In [26]:
def get_col(A: np.ndarray, cell: str):
    result = []
    for i in range(NROW):
        if cell == 'b':
            result.append(A[i].b)
        if cell == 'r':
            result.append(A[i].r)
        if cell == 'w':
            result.append(A[i].w)
        if cell == 'g':
            result.append(A[i].g)
    return result


In [27]:
# add constraints for column encoding
for r in range(total_r):
    for j in range(NCOL):
        S_col[r, j].u = gp.max_(get_col(S[r,:,j], 'w'))   # if one input cell is white, the output column is marked by u = 1 (unknown)
        S_col[r, j].x = gp.min_(get_col(S[r,:,j], 'b'))   # if all cells are known for fwd computation, marked by x = 1
        S_col[r, j].y = gp.min_(get_col(S[r,:,j], 'r'))   # if all cells are known for fwd computation, marked by y = 1

        M_col[r, j].u = gp.max_(get_col(M[r,:,j], 'w'))   # if one input cell is white, the output column is marked by u = 1 (unknown)
        M_col[r, j].x = gp.min_(get_col(M[r,:,j], 'b'))   # if all cells are known for fwd computation, marked by x = 1
        M_col[r, j].y = gp.min_(get_col(M[r,:,j], 'r'))   # if all cells are known for fwd computation, marked by y = 1

In [28]:
# generate MC rule, calculate the consumed df for both fwd and bwd
def gen_MC_rule(input: np.ndarray, input_col: ColumnEncode, output: np.ndarray, fwd: gp.Var, bwd: gp.Var):
    m.addConstr(NROW*input_col.u + gp.quicksum(get_col(output, 'b')) <= NROW)
    m.addConstr(gp.quicksum(get_col(input, 'b')) + gp.quicksum(get_col(output, 'b')) - NBRANCH*input_col.x <= 3)
    m.addConstr(gp.quicksum(get_col(input, 'b')) + gp.quicksum(get_col(output, 'b')) - 2*NROW*input_col.x >= 0)

    m.addConstr(NROW*input_col.u + gp.quicksum(get_col(output, 'r')) <= NROW)
    m.addConstr(gp.quicksum(get_col(input, 'r')) + gp.quicksum(get_col(output, 'r')) - NBRANCH*input_col.y <= 3)
    m.addConstr(gp.quicksum(get_col(input, 'r')) + gp.quicksum(get_col(output, 'r')) - 2*NROW*input_col.y >= 0)

    m.addConstr(gp.quicksum(get_col(input, 'b')) - NROW*input_col.x - fwd == 0)
    m.addConstr(gp.quicksum(get_col(input, 'r')) - NROW*input_col.y - bwd == 0)

In [29]:
# generate match rule, calculate the degree of matching of each column
def gen_match_rule(input: np.ndarray, output: np.ndarray, match: gp.Var):
    m.addConstr(match = gp.max_(0, (
        gp.quicksum(get_col(input, 'b')) + gp.quicksum(get_col(input, 'r')) - gp.quicksum(get_col(input, 'g')) +
        gp.quicksum(get_col(output, 'b')) + gp.quicksum(get_col(output, 'r')) - gp.quicksum(get_col(output, 'g')) - NROW))
    )

In [30]:
# set objective function: argmax obj = min{final_df_b, final_df_r, match_df}
def gen_objective():
    m.addConstr(final_df_b - gp.quicksum(start_df_b.flatten()) + gp.quicksum(consumed_fwd.flatten()) == 0)
    m.addConstr(final_df_r - gp.quicksum(start_df_r.flatten()) + gp.quicksum(consumed_bwd.flatten()) == 0)
    m.addConstr(match_df - gp.quicksum(match_col_df.flatten()) == 0)
    m.addConstr(obj - final_df_b <= 0)
    m.addConstr(obj - final_df_r <= 0)
    m.addConstr(obj - match_df <= 0)
    m.setObjective(obj, GRB.MAXIMIZE)

In [31]:
def formulation():
    # start round: no unknowns
    for i in range(NROW):
        for j in range(NCOL):
            m.addConstr(S[start, i, j].b + S[start, i, j].r >= 1)
            m.addConstr(start_df_b[i, j] + S[start, i, j].r == 1)
            m.addConstr(start_df_r[i, j] + S[start, i, j].b == 1)
    
    # intermediate rounds: 
    for r in range(1, total_r-1):
        nr = (r+1) % total_r
        if r in fwd:
            for j in range(NCOL):
                gen_MC_rule(input= M[r,:,j], input_col= M_col[r,j], output= S[nr,:,j], fwd= consumed_fwd[r], bwd= consumed_bwd[r])
        if r in bwd:
            for j in range(NCOL):
                gen_MC_rule(input= S[nr,:,j], input_col= S_col[nr,j], output= M[r,:,j], fwd= consumed_fwd[r], bwd= consumed_bwd[r])
    
    # last round:
    for j in range(NCOL):
        m.addConstr(consumed_fwd[total_r,j] == 0)
        m.addConstr(consumed_bwd[total_r,j] == 0)
        # jump the MC for last round
        for i in range(NROW):
            m.addConstr(M[total_r, i, j].b - S[0, i, j].b == 0)
            m.addConstr(M[total_r, i, j].r - S[0, i, j].r == 0)

    # match round:
    nmid = (mid + 1) % total_r
    for j in range(NCOL):
        gen_match_rule(input= M[mid,:,j], output= S[nmid,:,j], match= match_col_df)
        m.addConstr(consumed_fwd[mid, j] == 0)
        m.addConstr(consumed_bwd[mid, j] == 0)
    
    gen_objective()
    m.update()
    m.write(m.modelName + '.lp' )

In [32]:
def writeSol():
    if m.SolCount > 0:
        if m.getParamInfo(GRB.Param.PoolSearchMode)[2] > 0:
            gv = m.getVars()
            names = m.getAttr('VarName', gv)
            for i in range(m.SolCount):
                m.params.SolutionNumber = i
                xn = m.getAttr('Xn', gv)
                lines = ["{} {}".format(v1, v2) for v1, v2 in zip(names, xn)]
                with open('{}_{}.sol'.format(m.modelName, i), 'w') as f:
                    f.write("# Solution for model {}\n".format(m.modelName))
                    f.write("# Objective value = {}\n".format(m.PoolObjVal))
                    f.write("\n".join(lines))
        else:
            m.write(m.modelName + '.sol')
    else:
        print('infeasible')

In [33]:
formulation()
m.optimize()
writeSol()

TypeError: 'Var' object is not subscriptable